In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as spstats
import statsmodels.api as sm
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import altair as alt
from IPython.display import display

# Functions

In [ ]:
def check_dir(dir: str):
    """
    Creates a given path directory if it does not exist.

    Args:
        dir (str): Path to the directory to be created.
    """
    if os.path.exists(dir) and os.path.isdir(dir):
        pass
    else:
        os.makedirs(dir)


def dint_filter(results, alpha=0.05):
    """
    Filters results from dint_searcher function based on statistical significance and signal consistency.

    Args:
        results (pd.DataFrame): DataFrame containing correlation and regression analysis results.
        alpha (float, optional): Significance level for statistical tests. Defaults to 0.05.

    Returns:
        pd.DataFrame: Filtered results meeting the following criteria:
            - Both correlation and OLS p-values below alpha
            - Correlation and OLS coefficients have the same sign
            - Either tumour-normal p-values >= alpha OR different signal than tumour coefficient
    """
    # Both correlation and OLS p-values have to be lower than 0.05
    rhosign = results.rho_pval < alpha
    coefsign = results.coef_pval < alpha

    # Correlation and OLS have to have the same signal
    equalsignal = (results.rho/results.rho.abs()) == (results.coef/results.coef.abs())

    # Tumour-normal p-values have to be higher or equal to alfa or signal has to be different than tumour
    coefdiffnotsign = results.coef_diff_pval >= alpha
    distinctsign = (results.coef/results.coef.abs()) != (results.coef_diff/results.coef_diff.abs())

    return results[rhosign & coefsign & equalsignal & (coefdiffnotsign | distinctsign)].reset_index(drop=True)


def driver_neighbour_corr(
    driver: int,
    neighbourlist: list,
    mutationtab: np.array,
    expressiontab: np.array,
    ctfilter: np.array = None
) -> tuple:
    """
    Calculates Spearman correlation between a driver mutation and its neighbouring genes' expression.

    Parameters:
        driver (int): Index of the driver gene in the mutation table.
        neighbourlist (list): List of indices for neighbouring genes.
        mutationtab (np.array): 2D array of shape (n_samples, n_drivers) containing mutation data.
        expressiontab (np.array): 2D array of shape (n_samples, n_neighbours) containing expression data.
        ctfilter (np.array, optional): Boolean array of shape (n_drivers, n_samples) for filtering samples.
            Defaults to None, in which case all samples are used.

    Returns:
        tuple: Two elements:
            - rho: Array of correlation coefficients
            - pvalue: Array of corresponding p-values
    """
    if ctfilter is not None:
        filt = ctfilter[driver]
    else:
        filt = [True]*len(mutationtab)

    rho, pvalue = spstats.spearmanr(
        expressiontab[np.ix_(filt, neighbourlist)],
        mutationtab[filt, driver]
    )

    if len(neighbourlist) > 1:
        return rho[-1, :-1], pvalue[-1, :-1]
    else:
        return [rho], [pvalue]


def ols(y, X, resname):
    """
    Performs Ordinary Least Squares regression and returns coefficient and p-value for a specific variable.

    Args:
        y (array-like): Dependent variable.
        X (array-like): Independent variables (including constant term if needed).
        resname (str or int): Name/position of the variable whose coefficient is of interest.

    Returns:
        dict: Dictionary containing:
            - coef: Coefficient estimate for the specified variable
            - coef_pval: P-value for the coefficient
    """
    result = sm.OLS(y, X).fit()
    return {"coef": result.params[resname], "coef_pval": result.pvalues[resname]}


def driver_neighbour_ols(
    driver: int,
    neighbourlist: list,
    regressorstab: np.array,
    expressiontab: np.array,
    regressorsfilter: np.array = None,
    driverpos: int = 0,
    n_jobs: int = 1
):
    """
    Performs parallel OLS regression analysis between a driver mutation and its neighbouring genes' expression.

    Parameters:
        driver (int): Index of the driver gene.
        neighbourlist (list): List of indices for neighbouring genes.
        regressorstab (np.array): 2D array of shape (n_samples, n_regressors) containing predictor variables.
        expressiontab (np.array): 2D array of shape (n_samples, n_neighbours) containing expression data.
        regressorsfilter (np.array, optional): Boolean 2D array of shape (n_drivers, n_regressors) for filtering regressors.
        driverpos (int, optional): Position of the driver variable in the regressors table. Defaults to 0.
        n_jobs (int, optional): Number of parallel jobs to run. Defaults to 1.

    Returns:
        list: List of dictionaries containing regression results for each neighbour, including:
            - coef: Coefficient estimate
            - coef_pval: P-value for the coefficient
    """
    if regressorsfilter is not None:
        regfilt = regressorsfilter[driver]
        sampfilt = np.flatnonzero(regressorstab[:, regfilt].sum(axis=1)>1)
    else:
        regfilt = [True]*regressorstab.shape[1]
        sampfilt = [True]*regressorstab.shape[0]
        
    ols_list = Parallel(n_jobs=n_jobs)(delayed(ols)(
        expressiontab[sampfilt, neighbour],
        regressorstab[np.ix_(sampfilt, regfilt)],
        driverpos) for neighbour in neighbourlist)

    return ols_list


def dint_searcher(
    neighbourtab: pd.DataFrame,
    mutationtab: pd.DataFrame,
    tumourexp: pd.DataFrame,
    normalexp: pd.DataFrame,
    savetofile: str = False,
    filterdints: bool = False,
    alpha: float = 0.05,
    shuffle: np.random.Generator = False,
    progressbar: bool = False,
    n_jobs: int = 1
):
    """
    Searches for driver-neighbour interactions (DINTs) by analyzing relationships
    between driver mutations and neighbouring gene expression.

    Parameters:
        neighbourtab (pd.DataFrame): Boolean matrix indicating neighbourhood relationships between genes.
        mutationtab (pd.DataFrame): Matrix of mutation data with MultiIndex (patient, cancer_type).
        tumourexp (pd.DataFrame): Matrix of tumour expression data.
        normalexp (pd.DataFrame): Matrix of normal tissue expression data.
        savetofile (str, optional): Path to save results. If False, returns results instead. Defaults to False.
        filterdints (bool, optional): Whether to filter results using dint_filter. Defaults to False.
        alpha (float, optional): Significance level for statistical tests. Defaults to 0.05.
        shuffle (np.random.Generator, optional): Random number generator for permutation testing. Defaults to False.
        progressbar (bool, optional): Whether to show progress bar. Defaults to False.
        n_jobs (int, optional): Number of parallel jobs for OLS analysis. Defaults to 1.

    Returns:
        pd.DataFrame or None: If savetofile is False, returns DataFrame containing:
            - rho: Spearman correlation coefficient
            - rho_pval: Correlation p-value
            - driver: Driver gene identifier
            - neighbour: Neighbour gene identifier
            - coef: OLS coefficient
            - coef_pval: OLS p-value
            - coef_diff: Tumour-normal difference coefficient
            - coef_diff_pval: Tumour-normal difference p-value
        If savetofile is True, saves results to file and returns None.
    """
    if shuffle:
        # For correlation and tumour OLS, shuffle is made on cancer type
        mutationtab_ = mutationtab.copy()
        mutationtab_.index = pd.MultiIndex.from_tuples(
            shuffle.permutation(mutationtab.index), names=["patient", "cancer_type"])
    else:
        mutationtab_ = mutationtab.copy()

    # ------------------------- Preprocessing ------------------------------- #
    # correlation preprocessing
    # filter by cancer type
    # Since these filters will be index by driver, we will transpose the arrays
    # to simplify indexing
    ctfilter = mutationtab_.groupby("cancer_type").agg(lambda x: x.sum() > 0).T.to_numpy()
    ctfilterdiff = (
        mutationtab.loc[normalexp.index.get_level_values("patient")]
            .groupby("cancer_type").agg(lambda x: x.sum() > 0)
    ).T.to_numpy()

    # Calculate mean for spearman correlation analysis
    tumourexpmean = tumourexp.groupby("cancer_type").mean().to_numpy()
    mutationtabmean = mutationtab_.groupby("cancer_type").mean().to_numpy()

    # OLS preprocessing
    # Create cancer type dummy regressors for OLS analysis
    regressors = pd.get_dummies(mutationtab_.reset_index(level=1), dtype=int, drop_first=False)
    
    # diff will be shuffled in the end
    regressorsdiff = pd.get_dummies(
        mutationtab.loc[normalexp.index.get_level_values("patient")].reset_index(level=1),
            dtype=int, drop_first=False)
    
    # add constant column
    regressors = sm.add_constant(regressors, prepend=False).to_numpy()
    regressorsdiff = sm.add_constant(regressorsdiff, prepend=False).to_numpy()
    
    # create regressorsfilter to filter cancer_types and drivers in regressors array
    regressors_filter = np.concatenate([
        np.identity(mutationtab.shape[1]).astype(bool), # to choose Driver Mutation Profile
        np.concatenate([
            ctfilter, # choose only cancer types with at least one mutated patient
            np.ones((ctfilter.shape[0], 1)) # choose constant column for all drivers
        ], axis=1).astype(bool), ], axis=1)
    
    regressorsdiff_filter = np.concatenate([
        np.identity(mutationtab.shape[1]).astype(bool),
        np.concatenate([ctfilterdiff, np.ones((ctfilterdiff.shape[0], 1))], axis=1).astype(bool),
        ], axis=1)

    # calculate tumour - normal expression
    tumournormaldiff = (tumourexp.loc[normalexp.index] - normalexp).to_numpy()
    tumourexparr = tumourexp.to_numpy()

    if shuffle:
        # for diff OLS, shuffle is made on expression
        shuffle.shuffle(tumournormaldiff)
        
    # -------------------------- DINT analysis -------------------------------#
    neighbourlist = [(driver, np.flatnonzero(neighbourtab[driver])) for driver in neighbourtab.columns.tolist()]
    
    corr_results = {col: [] for col in ["rho", "rho_pval", "driver", "neighbour"]}
    ols_results = []
    olsdiff_results = []

    if progressbar:
        neighbourlist = tqdm(neighbourlist)
    else:
        neighbourlist = neighbourlist

    for i, (driver, neighbours) in enumerate(neighbourlist):

        rho, pval = driver_neighbour_corr(i, neighbours, mutationtabmean,
                                          tumourexpmean, ctfilter=ctfilter)
        corr_results["rho"].extend(rho)
        corr_results["rho_pval"].extend(pval)
        corr_results["driver"].extend([driver]*len(neighbours))
        corr_results["neighbour"].extend(neighbours)

        ols_list = driver_neighbour_ols(i, neighbours, regressors, tumourexparr,
                                        regressorsfilter=regressors_filter, driverpos=0, n_jobs=n_jobs)
        ols_results.extend(ols_list)

        ols_list = driver_neighbour_ols(i, neighbours, regressorsdiff, tumournormaldiff,
                                        regressorsfilter=regressorsdiff_filter, driverpos=0, n_jobs=n_jobs)

        olsdiff_results.extend(ols_list)

    # ------------------------------ Result Filtering ------------------------------#
    results = pd.merge(
        pd.DataFrame(corr_results).set_index("neighbour"),
        pd.Series(neighbourtab.index).rename("neighbour"), left_index=True, right_index=True
    ).reset_index(drop=True)

    results = pd.concat([results, pd.DataFrame(ols_results), pd.DataFrame(olsdiff_results).rename(columns={
        "coef": "coef_diff",
        "coef_pval": "coef_diff_pval"
    })], axis=1)

    if filterdints:
        results = dint_filter(results.dropna(), alpha=alpha)

    if savetofile:
        results.to_csv(savetofile, index=False)
    else:
        return results

# Directories

In [3]:
datadir = "data/"
shuffledir = datadir + "shuffle/"
check_dir(datadir)
check_dir(shuffledir)

In [4]:
networks = ["biogrid", "apid", "huri", "string", "omnipath"]

# Driver Neighbour analysis

## Load Datasets

In [5]:
mutationtab = pd.read_feather(datadir+"mutation.feather")
print(mutationtab.shape)
display(mutationtab.head(2))
neighbourtab = pd.read_feather(datadir+"neighbours.feather")
print(neighbourtab.shape)
display(neighbourtab.head(2))

(8404, 2570)


,,A1CF,A2ML1,ABCA10,ABCA13,ABCA7,ABCB1,ABCB5,ABCC3,ABCC5,ABCC9,...,ZPBP2,ZRANB3,ZRSR2,ZSCAN31,ZSCAN4,ZSWIM3,ZSWIM6,ZWILCH,ZWINT,ZZEF1
patient,cancer_type,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047,GBM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0055,GBM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(15206, 2570)


driver,A1CF,A2ML1,ABCA10,ABCA13,ABCA7,ABCB1,ABCB5,ABCC3,ABCC5,ABCC9,...,ZPBP2,ZRANB3,ZRSR2,ZSCAN31,ZSCAN4,ZSWIM3,ZSWIM6,ZWILCH,ZWINT,ZZEF1
neighbour,,,,,,,,,,,,,,,,,,,,,
A1BG,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A1CF,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
tumourexp = pd.read_feather(datadir+"tumour_expression.feather")
print(tumourexp.shape)
display(tumourexp.head(2))
normalexp = pd.read_feather(datadir+"normal_expression.feather")
print(normalexp.shape)
display(normalexp.head(2))

(8404, 15206)


,,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patient,cancer_type,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047,GBM,6.98,0.0,15.05,5.40,5.22,1.16,8.87,8.92,7.87,10.01,...,8.03,8.66,6.05,8.48,10.12,0.69,10.24,11.92,10.45,9.24
TCGA-02-0055,GBM,8.62,0.0,15.39,1.42,8.93,0.64,9.22,8.31,6.66,10.41,...,8.87,7.95,5.45,8.14,9.25,2.60,9.85,13.49,9.25,9.49


(665, 15206)


,,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patient,cancer_type,,,,,,,,,,,,,,,,,,,,,
TCGA-22-4593,LUSC,5.66,0.0,17.08,0.00,8.60,1.62,9.13,9.96,7.28,9.97,...,7.60,7.19,5.22,8.32,9.63,0.83,9.89,12.92,10.51,8.83
TCGA-22-4609,LUSC,6.12,0.0,16.80,2.28,8.59,0.76,8.83,9.60,6.89,9.41,...,7.29,5.59,6.04,9.02,10.24,1.03,9.87,12.66,11.12,9.19


In [9]:
tumourexp.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8404 entries, ('TCGA-02-0047', 'GBM') to ('TCGA-ZX-AA5X', 'CESC')
Columns: 15206 entries, A1BG to ZZZ3
dtypes: float64(15206)
memory usage: 975.3+ MB


## DINT Search

In [14]:
results = dint_searcher(
        neighbourtab, mutationtab, tumourexp, normalexp, savetofile=False,
        filterdints=False, shuffle=False, progressbar=True, n_jobs=-1
    )
results.to_csv(datadir+"results.csv", index=False)
results.head()

  0%|          | 0/2570 [00:00<?, ?it/s]

,rho,rho_pval,driver,neighbour,coef,coef_pval,coef_diff,coef_diff_pval
0,-0.242212,0.317756,A1CF,APOB,-0.307001,0.117268,-1.013937,0.355157
1,0.159719,0.513660,A1CF,APOBEC1,NaN,NaN,0.394227,0.398841
2,-0.018429,0.940308,A1CF,APOBEC2,0.094182,0.501902,0.413046,0.495099
3,0.258886,0.284517,A1CF,APOBEC3A,-0.043689,0.779266,-0.015337,0.981601
4,0.365950,0.123349,A1CF,APOBEC3B,0.041200,0.785462,0.535376,0.393236


## Shuffle

In [13]:
from random import getrandbits
seed = getrandbits(128)
print(seed)
rng = np.random.default_rng(seed)

309814212404889481310444358379219863528


In [14]:
for i, stream in enumerate(tqdm(rng.spawn(100))):

    dint_searcher(
        neighbourtab, mutationtab, tumourexp, normalexp, savetofile=shuffledir+f"shuffle{i+1}.csv",
        filterdints=True, alpha=0.05, shuffle=stream, progressbar=False, n_jobs=-1
    )

  0%|          | 0/100 [00:00<?, ?it/s]

/home/rnasysbiolab1/miniconda3/envs/jamiranda/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/rnasysbiolab1/miniconda3/envs/jamiranda/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


# Results

In [15]:
results = pd.read_csv(datadir+"results.csv")
results.head()

,rho,rho_pval,driver,neighbour,coef,coef_pval,coef_diff,coef_diff_pval
0,-0.242212,0.317756,A1CF,APOB,-0.307001,0.117268,-1.013937,0.355157
1,0.159719,0.513660,A1CF,APOBEC1,NaN,NaN,0.394227,0.398841
2,-0.018429,0.940308,A1CF,APOBEC2,0.094182,0.501902,0.413046,0.495099
3,0.258886,0.284517,A1CF,APOBEC3A,-0.043689,0.779266,-0.015337,0.981601
4,0.365950,0.123349,A1CF,APOBEC3B,0.041200,0.785462,0.535376,0.393236


In [16]:
resultsfiltered = dint_filter(results.dropna()).set_index(["driver", "neighbour"]).reset_index()
resultsfiltered.to_csv(datadir+"DINTs.csv", index=False)
resultsfiltered.head()

,driver,neighbour,rho,rho_pval,coef,coef_pval,coef_diff,coef_diff_pval
0,A2ML1,BAK1,0.616355,0.001340,0.099646,0.035045,-0.025126,0.910538
1,A2ML1,CEP152,0.446281,0.028816,0.233134,0.000372,0.161886,0.546178
2,A2ML1,DOT1L,0.441061,0.030974,0.171850,0.000543,0.139439,0.557473
3,A2ML1,SRRT,0.550239,0.005338,0.084226,0.005848,-0.069655,0.634963
4,A2ML1,ST6GALNAC6,-0.561549,0.004299,-0.135210,0.007368,0.238886,0.367315


In [17]:
print(resultsfiltered.driver.unique().shape[0])
print(resultsfiltered.neighbour.unique().shape[0])
print(len(resultsfiltered))

1655
3120
14197


# Shuffle Results

In [6]:
dints = pd.read_csv(datadir+"DINTs.csv", usecols=["driver", "neighbour"])
dints.head(2)

,driver,neighbour
0,A2ML1,BAK1
1,A2ML1,CEP152


In [7]:
rand_dist = [len(pd.read_csv(shuffledir+shuffle)) for shuffle in os.listdir(shuffledir)]
print(np.mean(rand_dist)/len(dints))

0.3696640135239839


In [8]:
common_dist = []
for shuffle in os.listdir(shuffledir):
    x = pd.read_csv(shuffledir+shuffle, usecols=["driver", "neighbour"])
    common_dist.append(x.merge(dints).shape[0]/x.shape[0])

common_dist = pd.DataFrame(common_dist, columns=["pct"])

In [9]:
(alt
 .Chart(common_dist, width=800, height=200, title="Percentage of DINTs found in Shuffled")
 .mark_boxplot(size=50, ticks=True)
 .encode(alt.X("pct:Q").axis(format='%'))
)

alt.Chart(...)

## New filter

In [15]:
alpha = 0.001
dints = pd.read_csv(datadir+"DINTs.csv")
newdints = dint_filter(dints, alpha=alpha)
print(len(newdints))

newrand_dist = [len(dint_filter(pd.read_csv(shuffledir+shuffle), alpha=alpha)) for shuffle in os.listdir(shuffledir)]

print(np.mean(newrand_dist)/len(newdints))

813
0.08460024600246002


In [16]:
newdints.to_csv(datadir+"DINTs0.001.csv", index=False)

In [17]:
alpha = 0.001
dints = dint_filter(pd.read_csv(datadir+"results_mac.csv"))
newdints_mac = dint_filter(dints, alpha=alpha)
print(len(newdints))

newrand_dist = [len(dint_filter(pd.read_csv(shuffledir+shuffle), alpha=alpha)) for shuffle in os.listdir(shuffledir)]

print(np.mean(newrand_dist)/len(newdints_mac))

813
0.08428921568627451


In [18]:
newdints[["driver", "neighbour"]].merge(newdints_mac[["driver", "neighbour"]], how="inner")

,driver,neighbour
0,AFF3,TTF2
1,AHI1,LYAR
2,AHNAK,EFTUD2
3,AKAP9,CEP55
4,AKAP9,TUB
...,...,...
790,ZMYM3,MTA2
791,ZNF217,EZH2
792,ZNF536,CENPA
793,ZNF668,ZNF512
